In [1]:
%matplotlib inline

import numpy as np
# import cupy as cp
import matplotlib.pyplot as plt
from scipy.signal import convolve2d
from skimage import data, io, color
from skimage.transform import resize
from pandas import DataFrame

plt.rcParams['figure.figsize'] = [10, 10]

def show_image(image, title, flip_x_axis=False):
    if flip_x_axis:
        image = np.fliplr(image)
    plt.imshow(image, cmap=plt.get_cmap("gray"))
    plt.title(title)
    plt.colorbar()
    plt.show()
    
def normalise(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))

def rrmse(observed, ideal, decimal=6):
    return "{:.{}f}".format(np.sqrt((1 / observed.shape[0]**2) * np.sum((observed-ideal)**2) / np.sum(ideal**2)) * 100.0, decimal)

def laplacian_of_gaussian(x, y, sigma):
    p = (x**2.0 + y**2.0) / 2.0 * sigma**2.0
    return -(1.0 / (np.pi * sigma**4.0)) * (1.0 - p) * np.exp(-p)

def decimation_matrix(l, m):
    d_matrix = np.zeros((m**2, l**2), dtype=np.float32)

    tile = np.repeat((1, 0, 1), (2, l - 2, 2)) # assuming taking 2 neighbours per dimension
    t_len = tile.shape[0]
    d = l // m
    r_offset = m**2 // 2
    c_offset = l**2 // 2

    for p in np.arange(l//4): # divide by 4 as 4 neighbours total
        p_offset = p * l
        for q in np.arange(m):
            d_matrix[q+ p_offset//2, q*d + p_offset*2 : q*d+t_len + p_offset*2] = tile # top-left quadrant
            d_matrix[q+r_offset + p_offset//2, q*d+c_offset + p_offset*2: q*d+t_len+c_offset + p_offset*2] = tile # bottom-right quadrant
    return d_matrix

# produces convolution matrix of size l**2 by l**2, where each row is populated by the convolution kernel values at the appropriate neighbours
# note: assumes kernel is a two-dimensonal numpy array of some size n by n
def convolution_matrix(l, kernel):
    
    conv = np.zeros((l**2, l**2), dtype=np.float32)
    full_supp = kernel.shape[0] # assumed square
    half_supp = (full_supp - 1) // 2

    for conv_row in np.arange(l**2):

        row, col = (conv_row // l, conv_row % l)

        for k_row in np.arange(-(half_supp), half_supp + 1):
            # map "kernel row" to rows in conv
            mapped_row = row + k_row
            # ignore any out of bounds rows
            if mapped_row >= 0 and mapped_row < l:
                linear_col = col - half_supp
                # truncate negative columns
                mapped_col_start = max(linear_col, 0)
                # truncate columns which exceed the l dimension
                mapped_col_end = min(linear_col + full_supp, l)
                # left trimming for kernels when overlapping out of bounds region in conv (col < 0)
                left = np.absolute(col - half_supp) if linear_col < 0 else 0
                # right trimming for kernels when overlapping out of bounds region in conv (col >= l)
                right = linear_col + full_supp - l if linear_col + full_supp >= l else 0 
                # copy over kernel row for current k_row, possibly including trimming for out of bounds coordinates
                conv[conv_row][mapped_row * l + mapped_col_start : mapped_row * l + mapped_col_end] = kernel[k_row + half_supp][left: left + full_supp - right]
    return conv

#### Configuration and data set up...

In [2]:
dataframe = DataFrame()

timesteps = 30 # total timesteps
timesteps_per_y = 5
l = 100
m = 50
n = timesteps // timesteps_per_y
w = np.ones(n)

half_supports = np.arange(2, 27) # 3 .. 51
sigmas = np.linspace(0.5, 10.0, 40) # strides of 0.1
betas = [1.631635943, 1.024517513, 4.13838621, 10.49636134, 12.25763632, 57.82109646, 67.52339687,	
         92.08527729, 107.5370601, 0.877306662, 0.877306662, 0.751248241, 0.751248241, 0.751248241,	
         0.6433029, 0.6433029, 0.550868007, 0.550868007, 0.550868007, 0.550868007, 0.471714896,
         0.253634166, 0.253634166, 0.217189985, 0.217189985]

dataframe["Sigmas"] = sigmas

filename = "../data/direct_image_ts_0_29_800x800.bin"
x_true = np.fromfile(filename, dtype=np.float32)
x_true = resize(x_true.reshape(800, 800), (l, l), anti_aliasing=False, order=1)
x_true = normalise(x_true)

filename = "../data/direct_psf_ts_0_29_800x800.bin"
x_psf = np.fromfile(filename, dtype=np.float32).reshape(800, 800)[1:, 1:]
x_psf = resize(x_psf, (l-1, l-1), anti_aliasing=False, order=1)
x_psf = np.pad(x_psf, ((1, 0), (1, 0))) # pad with new 0th row/col to ensure trimming from centre

# Storing all low-res images as layered stack
y = np.zeros((n, m, m))

# batched time steps direct images
for i in np.arange(n):
    filename = f"../data/direct_image_ts_{i * timesteps_per_y}_{i * timesteps_per_y + timesteps_per_y - 1}.bin"
    y[i] = np.fromfile(filename, dtype=np.float32).reshape(m, m)
    y[i] = normalise(y[i])

# Decimation matrix
d = decimation_matrix(l, m)

for idx, half_supp in enumerate(half_supports):
    errors = []
    for σ in sigmas:
        
        β = betas[idx]

        print(f"σ = {σ}, full support = {half_supp * 2 - 1}, β = {β}") 

        psf_min = l//2 - (half_supp - 1)
        psf_max = l//2 + half_supp
        x_psf_trim = x_psf.copy()[psf_min:psf_max, psf_min:psf_max]
        x_psf_trim /= np.sum(x_psf_trim)

        # Blur matrix (psf)
        h = convolution_matrix(l, x_psf_trim)

        # Sharpening matrix (laplacian of gaussian)
        psf_central_support = 15 # num pixels around centre of PSF which represent approx 1/3 of the curve, maybe a bit bigger for padding
        downsampling = np.linspace(-(psf_central_support-1)//2, (psf_central_support-1)//2, num=psf_central_support)
        lap_of_gauss = np.zeros((downsampling.shape[0], downsampling.shape[0]), np.float32)
        lap_gauss_σ = σ # need to play around with this to get similar structure to true laplacian 3x3
        for i in np.arange(downsampling.shape[0]):
            for j in np.arange(downsampling.shape[0]):
                lap_of_gauss[i][j] = -laplacian_of_gaussian(downsampling[i], downsampling[j], lap_gauss_σ)

        lap_of_gauss /= np.max(lap_of_gauss)
        s = convolution_matrix(l, lap_of_gauss)
        β *= 16.0
        
        b = np.zeros(l**2, dtype=np.float32)

        for i in np.arange(n):
            b += np.matmul(w[i] * h.T, np.matmul(d.T, y[i].flatten()))

        lhs = β * np.matmul(s.T, s)
        rhs = (h.T @ d.T @ d @ h) * np.sum(w)
        a = lhs + rhs

        x = np.linalg.solve(a, b)
        x = x.reshape(100, 100)

        errors.append(rrmse(normalise(x), normalise(x_true)))

        if σ == sigmas[-1]:
            dataframe[f"{half_supp * 2 - 1}x{half_supp * 2 - 1} (β={β/16.0}) RRMSE"] = errors
            
# Finally, write the results to a spreadsheet...
dataframe.to_excel('../lap_of_gaussian_16x_beta.xlsx', sheet_name='16x Beta', index=False)

σ = 0.5, full support = 3, β = 1.631635943
σ = 0.7435897435897436, full support = 3, β = 1.631635943
σ = 0.9871794871794872, full support = 3, β = 1.631635943
σ = 1.2307692307692308, full support = 3, β = 1.631635943
σ = 1.4743589743589745, full support = 3, β = 1.631635943
σ = 1.7179487179487178, full support = 3, β = 1.631635943
σ = 1.9615384615384615, full support = 3, β = 1.631635943
σ = 2.2051282051282053, full support = 3, β = 1.631635943
σ = 2.448717948717949, full support = 3, β = 1.631635943
σ = 2.692307692307692, full support = 3, β = 1.631635943
σ = 2.9358974358974357, full support = 3, β = 1.631635943
σ = 3.1794871794871793, full support = 3, β = 1.631635943
σ = 3.423076923076923, full support = 3, β = 1.631635943
σ = 3.6666666666666665, full support = 3, β = 1.631635943
σ = 3.91025641025641, full support = 3, β = 1.631635943
σ = 4.153846153846153, full support = 3, β = 1.631635943
σ = 4.397435897435898, full support = 3, β = 1.631635943
σ = 4.6410256410256405, full support

In [3]:
print("Finished")

Finished
